In [31]:
# ---------------------------------
# Import requirement modules
# ---------------------------------
import xlwings as xw
import pandas as pd
import numpy as np
import time
import datetime as dt
import requests
import json
import urllib3
from pathlib import Path


In [32]:
# -----------------------------------------
# Some Initial variable for the application
# -----------------------------------------
numberOfRecordsInEachFile = 100
fileNameStartWith = 'AMG-'


In [33]:
def createINITrecord(INIT_BPKIND, INIT_BU_GROUP, INIT_BU_RLTYP, INIT_BU_TYPE, INIT_MUSTER_KUN, INIT_PARTNER):
    v_INIT_record_formatted = f'{INIT_BPKIND}\t{INIT_BU_GROUP}\t{INIT_BU_RLTYP}\t{INIT_BU_TYPE}\t{INIT_MUSTER_KUN}\t{INIT_PARTNER}'
    return v_INIT_record_formatted

In [34]:
def createBUT000record(INIT_BPKIND, INIT_BU_GROUP, INIT_BU_RLTYP, INIT_BU_TYPE, INIT_MUSTER_KUN, INIT_PARTNER):
    v_INIT_record_formatted = f'{INIT_BPKIND}\t{INIT_BU_GROUP}\t{INIT_BU_RLTYP}\t{INIT_BU_TYPE}\t{INIT_MUSTER_KUN}\t{INIT_PARTNER}'
    return v_INIT_record_formatted

In [35]:
def createBUT020record(INIT_BPKIND, INIT_BU_GROUP, INIT_BU_RLTYP, INIT_BU_TYPE, INIT_MUSTER_KUN, INIT_PARTNER):
    v_INIT_record_formatted = f'{INIT_BPKIND}\t{INIT_BU_GROUP}\t{INIT_BU_RLTYP}\t{INIT_BU_TYPE}\t{INIT_MUSTER_KUN}\t{INIT_PARTNER}'
    return v_INIT_record_formatted

In [36]:
def createBUT0IDrecord(INIT_BPKIND, INIT_BU_GROUP, INIT_BU_RLTYP, INIT_BU_TYPE, INIT_MUSTER_KUN, INIT_PARTNER):
    v_INIT_record_formatted = f'{INIT_BPKIND}\t{INIT_BU_GROUP}\t{INIT_BU_RLTYP}\t{INIT_BU_TYPE}\t{INIT_MUSTER_KUN}\t{INIT_PARTNER}'
    return v_INIT_record_formatted

In [37]:
def createENDLINErecord(INIT_BPKIND, INIT_BU_GROUP, INIT_BU_RLTYP, INIT_BU_TYPE, INIT_MUSTER_KUN, INIT_PARTNER):
    v_INIT_record_formatted = f'{INIT_BPKIND}\t{INIT_BU_GROUP}\t{INIT_BU_RLTYP}\t{INIT_BU_TYPE}\t{INIT_MUSTER_KUN}\t{INIT_PARTNER}'
    return v_INIT_record_formatted

In [38]:
# ---------------------------------
# Initialize workbook by reading the existing one
# ---------------------------------
wb = xw.Book(r'DATA-Emigall\Emigall-BP-Source-Data.xlsx') 
# Separate and hidden Excel instance
app = xw.App(visible=False)  
# --------------------------------------------------
# Reading Tab: DATATAB_1 and getting total row count
# --------------------------------------------------
data_info_sheet = wb.sheets['DATATAB_1']
#this just does ctrl+shift+down
lastcell = data_info_sheet.range(1,1).end('down').row 
#just so you know how many rows you have.
print('Total Number of rows: {}'.format(lastcell)) 
wb.close()

Total Number of rows: 38


In [39]:
# --------------------------------------------------------------------------
# Determine number of to be generated based on Number of record in each file 
# and actual number of records in excel sheet
# --------------------------------------------------------------------------
def determineNumberOfFiles(i_numberOfRecordsInEachFile, i_lastcell):
    o_numberOfFiles2Generate = (i_lastcell % i_numberOfRecordsInEachFile)
    l_quotient  = i_lastcell // i_numberOfRecordsInEachFile
    l_mod = i_lastcell % i_numberOfRecordsInEachFile
    if(l_mod > 0):
        o_numberOfFiles2Generate = l_quotient + 1
    else: 
        o_numberOfFiles2Generate = l_quotient
    return o_numberOfFiles2Generate

In [40]:
numberOfFiles2Generate = determineNumberOfFiles(numberOfRecordsInEachFile, lastcell)
print('Total Number of files to be generated: {}'.format(numberOfFiles2Generate)) 

Total Number of files to be generated: 1


In [41]:
def extractDataChunkForFile(fromCount, toCount):
    wb = None
    app = None
    data_sheet = None
    df1 = None
    
    wb = xw.Book(r'DATA-Emigall\Emigall-BP-Source-Data.xlsx') 
    # Start invisible Excel
    app = xw.App(visible=False, add_book=False)
    data_sheet = wb.sheets['DATATAB_1']
    # Reading from Excel as Dataframe
    df1 = data_sheet.range((fromCount,1),(toCount,11)).value
    wb.close()
    return df1

In [42]:
xdf1 = extractDataChunkForFile(1,5)
print(len(xdf1))
# looping
for x in range(0, len(xdf1), 1):
    lineToPrint = f'{xdf1[x][0]}\t{xdf1[x][1]}\t{xdf1[x][2]}\t{xdf1[x][3]}\t{xdf1[x][4]}\t{xdf1[x][5]}'
    # print(xdf1[x][0])
    print(lineToPrint)

5
EXT_ID	INIT_BPKIND	INIT_BU_GROUP	INIT_BU_RLTYP	INIT_BU_TYPE	INIT_MUSTER_KUN
PY001	MCST	OLSM	None	2.0	None
PY002	MCST	OLSM	None	2.0	None
PY003	MCST	OLSM	None	2.0	None
PY004	MCST	OLSM	None	2.0	None


In [43]:
# ***********************************************************************
# The parameter and it's description
# fileName - Tells what should be the file name for the program to create
# fromCount - It's the record number where it should start from in the input 
#             file (in this case it will be the Excel row)
# toCount - record number where it should stop of that iteration
# ***********************************************************************
def createDataFileIfNotExists(fileName, iterationCount):
    
    # Calculate fromCount & toCount based on the iteration number
    
    fromCount = None
    toCount = None
    
    if (numberOfRecordsInEachFile >= lastcell):
        fromCount = 1
        toCount = lastcell
    else:
        if (iterationCount == 1):
            fromCount = 1
            toCount = numberOfRecordsInEachFile
        elif (iterationCount == numberOfFiles2Generate):
            fromCount = (numberOfRecordsInEachFile * (iterationCount - 1)) + 1
            toCount = lastcell
        else:
            fromCount = (numberOfRecordsInEachFile * (iterationCount - 1) + 1)
            toCount = (fromCount + numberOfRecordsInEachFile) -1
            
    print('The current count block is: {} & {} and I am in iteration {} & number of file to generate is: {}'.format(fromCount,toCount,iterationCount,numberOfFiles2Generate)) 
    
    
    f_content_new_line = "\n"
    with open(fileName,'w') as f:
        
        xdf1 = extractDataChunkForFile(fromCount+1,toCount)
        print(len(xdf1))
        # looping
        for x in range(0, len(xdf1), 1):
            lineToPrint = f'{xdf1[x][0]}\t{xdf1[x][1]}\t{xdf1[x][2]}\t{xdf1[x][3]}\t{xdf1[x][4]}\t{xdf1[x][5]}'
            f_content = lineToPrint
            f.write(f_content)
            f.write(f_content_new_line)
            f_content = "Dipankar Biswas Test 2"
            f.write(f_content)
            f.write(f_content_new_line)
            f_content_INIT = createINITrecord('INIT', xdf1[x][0], xdf1[x][1], xdf1[x][2], xdf1[x][3], xdf1[x][4])
            f.write(str(f_content_INIT))
            f.write(f_content_new_line)
            f_content_BUT000 = createBUT000record('BUT000', xdf1[x][0], xdf1[x][1], xdf1[x][2], xdf1[x][3], xdf1[x][4])
            f.write(str(f_content_BUT000))
            f.write(f_content_new_line)
            f_content_BUT020 = createBUT020record('BUT020', xdf1[x][0], xdf1[x][1], xdf1[x][2], xdf1[x][3], xdf1[x][4])
            f.write(str(f_content_BUT020))
            f.write(f_content_new_line)
            f_content_BUT0ID = createBUT0IDrecord('BUT0ID', xdf1[x][0], xdf1[x][1], xdf1[x][2], xdf1[x][3], xdf1[x][4])
            f.write(str(f_content_BUT0ID))
            f.write(f_content_new_line)
            f_content_ENDE = createENDLINErecord('&ENDE', xdf1[x][0], xdf1[x][1], xdf1[x][2], xdf1[x][3], xdf1[x][4])
            f.write(str(f_content_ENDE))
            f.write(f_content_new_line)
        return f.mode

In [44]:
# ---------------------------------
# Program logic:
# 1. Create the function which is responsible for creating the data file with a file name being passed.
# 2. Create a function which is responsible for creating the Header data into the data file.
# 3. Create a function which will create the Line items for BUT000
# 4. Create a function which will create the Line items for BUT0ID
# 5. Create a function which will create the Line items for Ending line item.
# ---------------------------------
# Call this file creation in a loop based on variable: numberOfFiles2Generate
# Dynamically determine number of file to be generated by first reading all the record count from the file and then 
# divide it by number defined in variable numberOfRecordsInEachFile.

# myResponse = createDataFileIfNotExists('DATA-Emigall\emigall-1.txt')
# Looping ...
# range(start, stop, step)
#

for x in range(0, numberOfFiles2Generate, 1):
    myResponse = createDataFileIfNotExists('DATA-Emigall\{}emigall-{}.txt'.format(fileNameStartWith, x+1), x+1)
    print(myResponse)

The current count block is: 1 & 38 and I am in iteration 1 & number of file to generate is: 1
37
w


In [45]:
# ---------------------------------
# Initialize workbook by reading the existing one
# ---------------------------------
wb = xw.Book(r'DATA-Emigall\Emigall-BP-Source-Data.xlsx') 
app = xw.App(visible=False)  # Separate and hidden Excel instancetable_1_sheet = wb.sheets['DATATAB_1']
# ---------------------------------
# Reading Tab: DATATAB_1
# ---------------------------------
data_sheet = wb.sheets['DATATAB_1']
# Reading from Excel as Dataframe
df1 = data_sheet['A1'].expand().options(pd.DataFrame, chunksize=1000).value
# df1
wb.close()